# Head Selection
**_BERT_** is a **_Multi-layer_ _Multi-Head_** Transformer architecture. As discuss in many of the current reseachers, different Attention heads captures different lingustic patterns. For a better deletion of words using Attention mechanism we need to choose a head which **captures pattern useful for classification.**

To do this we are using a Brute force mechanism to seach through all the possible heads. We are deleting TopK words attended by different heads from the sentence and measuring the new classification score. In case of sentiments, removing sentiments related words makes the sentence neutral. The heads are sorted by the amount to which it is able to make the sentences from dev set to Neutral.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 10.7 MB/s eta 0:00:00


In [ ]:
%cd "/content/drive/MyDrive/transformer-drg-style-transfer-master"

/content/drive/.shortcut-targets-by-id/1TCvtUjcTYKWgalUeZbJk_aFhLp1aLjr6/transformer-drg-style-transfer-master


In [ ]:
import csv
import logging
import os
import random
import sys
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear

from bertviz.bertviz import attention, visualization
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.0 MB/s eta 0:00:00


In [ ]:
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    RobertaModel,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    AdamW)

In [ ]:
logger = logging.getLogger(__name__)
bert_classifier_model_dir = "/content/drive/MyDrive/transformer-drg-style-transfer-master/robertaOutput/roberta.pt" ## Path of BERT classifier model path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
logger.info("device: {}, n_gpu {}".format(device, n_gpu))

In [ ]:
## Model for performing Classification
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model_dict = torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/robertaOutput/roberta.pt', map_location=device)
model_cls = RobertaForSequenceClassification.from_pretrained(pretrained_model_name_or_path='roberta-base', state_dict=model_dict)
model_cls.to(device)
model_cls.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
## Model to get the attention weights of all the heads
model_dict = torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/robertaOutput/roberta.pt', map_location=device)
model = RobertaModel.from_pretrained(pretrained_model_name_or_path='roberta-base', state_dict=model_dict, add_cross_attention=True, is_decoder = True)

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model.to(device)
model.eval()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [ ]:
max_seq_len=70 # Maximum sequence length 
sm = torch.nn.Softmax(dim=-1) ## Softmax over the batch

In [ ]:
def run_multiple_examples(input_sentences, bs=32):
    """
    This fucntion returns classification predictions for batch of sentences.
    input_sentences: list of strings
    bs : batch_size : int
    """


    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    ids_for_decoding = []

    roberta_input_ids = []
    roberta_attention_masks = []
    sentence_ids = []
    counter = 0
    pred_lt = []

    for sen in input_sentences:
      roberta_encoded_dict = roberta_tokenizer.encode_plus(
                        sen,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
      roberta_input_ids.append(roberta_encoded_dict['input_ids'])
      roberta_attention_masks.append(roberta_encoded_dict['attention_mask'])
      ids_for_decoding.append(roberta_encoded_dict['input_ids'])
      sentence_ids.append(counter)
      counter  = counter + 1
    roberta_input_ids = torch.cat(roberta_input_ids, dim=0)
    roberta_attention_masks = torch.cat(roberta_attention_masks, dim=0)
    roberta_dataset = TensorDataset(roberta_input_ids, roberta_attention_masks)
    roberta_train_dataloader = DataLoader(
            roberta_dataset,  # The training samples.
            sampler = RandomSampler(roberta_dataset), # Select batches randomly
            batch_size = 32 # Trains with this batch size.
        )
    for step, batch in enumerate(roberta_train_dataloader):
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      with torch.no_grad():
        logits = model_cls(b_input_ids, b_input_mask).logits
        preds = sm(logits)
        print(logits.size(), preds.size())
      pred_lt.extend(preds.tolist())
    return pred_lt

In [ ]:
def read_file(path,size):
    with open(path) as fp:
        data = fp.read().splitlines()[:size]
    return data

In [ ]:
def get_attention_for_batch(input_sentences , bs=32):
    """
    This function calculates attention weights of all the heads and
    returns it along with the encoded sentence for further processing.
    
    input sentence: list of strings
    bs : batch_size
    """
    
    ## Preprocessing for BERT 
    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    ids_for_decoding = []

    roberta_input_ids = []
    roberta_attention_masks = []
    sentence_ids = []
    counter = 0
    all_attn_probs = []

    for sen in input_sentences:
      roberta_encoded_dict = roberta_tokenizer.encode_plus(
                        sen,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
      roberta_input_ids.append(roberta_encoded_dict['input_ids'])
      roberta_attention_masks.append(roberta_encoded_dict['attention_mask'])
      ids_for_decoding.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("<s>" + sen + "</s>")))
      sentence_ids.append(counter)
      counter  = counter + 1
    roberta_input_ids = torch.cat(roberta_input_ids, dim=0)
    roberta_attention_masks = torch.cat(roberta_attention_masks, dim=0)
    roberta_dataset = TensorDataset(roberta_input_ids, roberta_attention_masks)
    roberta_train_dataloader = DataLoader(
            roberta_dataset,  # The training samples.
            sampler = RandomSampler(roberta_dataset), # Select batches randomly
            batch_size = 32 # Trains with this batch size.
        )
    for step, batch in enumerate(roberta_train_dataloader):
      batch_attn_probs = []
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      with torch.no_grad():
        attn = model(b_input_ids, b_input_mask, output_attentions=True).cross_attentions
      attn = list(attn)
      # for k in range(len(attn)):
      #   # attn[k] = list(attn[k])
      #   attn[k] = attn[k].to('cpu')
      # # print(len(attn_b[0]), len(attn_b[0][0]), len(attn_b[0][0][0]), len(attn_b[0][0][0][0]))
      # if step == 0:
      #   att_lt = attn
      #   heads = len(att_lt)
      #   # print((att_lt[0]))
      # else:
      #   for j in range(12):
      #       att_lt[j] = torch.cat((att_lt[j], attn[j]),0)
      
      for layer in attn:
        # print(layer[0].size()) #([32, 12, 128, 64]
        batch_attn_probs.append(layer[0].detach().cpu().unsqueeze(1))
      batch_attn_probs = torch.cat(batch_attn_probs, dim=1)  # Shape: (batch_size, num_heads, sequence_length, sequence_length)
      all_attn_probs.append(batch_attn_probs)

    # Stack the attention weights across all batches
    all_attn_probs = torch.cat(all_attn_probs, dim=0)  # Shape: (total_data, num_heads, sequence_length, sequence_length)

    # Transpose to get the required shape
    # Shape: (num_layers, sequence_length, total_data, num_heads, sequence_length)

    all_attn_probs = all_attn_probs.transpose(0, 1)  # Shape: (num_layers, num_heads, total_data, sequence_length, sequence_length)
    # return att_lt, ids_for_decoding
    return all_attn_probs, ids_for_decoding
    
    # return att_lt, ids_for_decoding

In [ ]:
def process_sentences(input_sentences, att, decoding_ids, threshold=0.25):
    """
    This function processes each input sentence by removing the top tokens defined threshold value.
    Each sentence is processed for each head.
    
    input_ids: list of strings
    decoding_ids: indexed input_sentnces thus len(input_sentences) == len(decoding_ids)
    threshold: Percentage of the top indexes to be removed
    """
    # List of None of num_of_layers * num_of_heads to save the results of each head for input_sentences
    
    lt = [None for x in range(len(att) * len(att[0][0]))]
    
    
    inx = 0
    for i in trange(len(att)): #  For all the layers
        for j in range(len(att[i][0])): # For all the heads in the ith Layer
            processed_sen = [None for q in decoding_ids] # List of len(decoding_ids)
            for k in range(len(input_sentences)): # For all the senteces 
                _, topi = att[i][k][j][0].topk(len(decoding_ids[k])) # Get top attended ids
                topi = topi.tolist()
                topi = topi[:int(len(topi) * threshold)] 
                print(topi)
                ## Decode the sentece after removing the topk indexes
                final_indexes = []
                count = 0
                count1 = 0
                # tokens = ["[CLS]"] + tokenizer.tokenize(input_sentences[k]) + ["[SEP]"]
                roberta_encoded_dict = roberta_tokenizer.encode_plus(
                        input_sentences[k],                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
                tokens = roberta_encoded_dict['input_ids']
                # print(decoding_ids[k], tokens[0][0])
                while count < len(decoding_ids[k]):
                    if count in topi: # Remove index if present in topk
                        while (count + count1 + 1) < len(decoding_ids[k]):
                            if "##" in str(tokens[0][count + count1 + 1]):
                                count1 += 1
                            else:
                                break
                        count += count1
                        count1 = 0
                        # continue
                        # pass
                    else: # Else add to the decoded sentence
                        final_indexes.append(decoding_ids[k][count])
                    count += 1
                tmp = tokenizer.convert_ids_to_tokens(final_indexes[0]) # Convert ids to token
                # Convert toknes to sentence
                processed_sen[k] = " ".join(tmp).replace('Ġ', '').replace(" ##", "").replace("<s>","").replace("</s>","").replace("<pad>","").strip()
            lt[inx] = processed_sen # Store sentences for inxth head
            inx += 1
    
    return lt

In [ ]:
def get_block_head(processed_sentence_list, lmbd = 0.1):
    """
    This function calculate classification scores for sentences generated by each head
    and sort them from best to worst.
    score = min(pred) + lmbd / max(pred) + lmbd, lmbd is smoothing param
    pred is list of probability score for each class, for best case pred = [0.5, 0.5] ==> score = 1
    
    it returns sorted list of (Layer, Head, Score)
    """
    scores = {}
    #scores_1 = {}
    for i in trange(len(processed_sentence_list)): # sentences by each head
        pred = np.array(run_multiple_examples(processed_sentence_list[i]))
        scores[i] = np.mean([(min(x[0], x[1])+lmbd)/(max(x[0], x[1])+lmbd) for x in pred])
        #scores_1[i] = np.mean([abs(max(x[0],x[1]) - min(x[0],x[1])) for x in pred])
    temp = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    #temp1 = sorted(scores_1.items(), key=lambda kv: kv[1], reverse=False)
    score_lt = [(x // 12, x - (12 * (x // 12)),y) for x,y in temp]
    #score1_lt = [(x // 12, x - (12 * (x // 12)),y) for x,y in temp1]
    return score_lt  #score1_lt

In [ ]:
pos_examples_file = "/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/sentiment.dev.1"
neg_examples_file = "/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/sentiment.dev.0"

In [ ]:
'''
100 examples from each class worked good, the bottlenack is the run_multiple_examples() function,
with higher memory (either with cpu of gpu) one can reduce the processing time by incresing batch_size.
With batch_size of 32 it takes around 24 mins for 100 example on cpu.
'''
pos_data = read_file(pos_examples_file,100)
neg_data = read_file(neg_examples_file,100)
data = pos_data + neg_data

In [ ]:
print(len(pos_data), len(neg_data), len(data))

100 100 200


In [ ]:
att, decoding_ids = get_attention_for_batch(data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
print(len(att), len(att[0]), len(att[0][0]),len(att[0][0][0]), len(att[0][0][0][0]))

12 200 12 128 64


In [ ]:
sen_list = process_sentences(data, att, decoding_ids)

In [ ]:
scores = get_block_head(sen_list)

In [ ]:
scores

[(8, 3, 0.23401291897378357),
 (7, 9, 0.23401291830588036),
 (10, 3, 0.23401291595791934),
 (1, 7, 0.21455849389106824),
 (3, 6, 0.21455849346674674),
 (7, 1, 0.21455849223055182),
 (11, 8, 0.2145584855443996),
 (6, 10, 0.20277125066330964),
 (8, 5, 0.20277125066330964),
 (0, 0, 0.18875358972700407),
 (0, 1, 0.18875358972700407),
 (0, 2, 0.18875358972700407),
 (0, 3, 0.18875358972700407),
 (0, 4, 0.18875358972700407),
 (0, 5, 0.18875358972700407),
 (0, 6, 0.18875358972700407),
 (0, 7, 0.18875358972700407),
 (0, 8, 0.18875358972700407),
 (0, 9, 0.18875358972700407),
 (0, 10, 0.18875358972700407),
 (0, 11, 0.18875358972700407),
 (1, 0, 0.18875358972700407),
 (1, 1, 0.18875358972700407),
 (1, 2, 0.18875358972700407),
 (1, 3, 0.18875358972700407),
 (1, 4, 0.18875358972700407),
 (1, 5, 0.18875358972700407),
 (1, 6, 0.18875358972700407),
 (1, 8, 0.18875358972700407),
 (1, 9, 0.18875358972700407),
 (1, 10, 0.18875358972700407),
 (1, 11, 0.18875358972700407),
 (2, 0, 0.18875358972700407),
 (2,